In [1]:
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from matplotlib.colors import ListedColormap

import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets
from pandas.plotting import scatter_matrix
import seaborn as sns
import matplotlib.patches as patches
from cv2 import cv2 as cv
import numpy as np
from imutils import contours
from matplotlib import pyplot as plt
from functools import cmp_to_key
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import os, os.path
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.manifold import TSNE
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
import cv2 as cv
import numpy as np


def prepareHistData(img, dst):
    # generate histogram
    hist, bins = np.histogram(img.ravel(), 256, [0, 256])

    # scale it to fit the shape of "canvas"
    maximum = hist.max()
    if maximum == 0:
        maximum = 1
    hist = hist / maximum * dst.shape[0]

    # now append X axis by stacking transposed histogram and linspace
    stack = np.vstack((np.linspace(0, dst.shape[1], 256), hist.reshape(-1))).T

    # this data is still flipped; unflip it
    stack[:, 1] = dst.shape[0] - stack[:, 1]

    return stack


def plotBinHistogram(img, dst, color=(0, 255, 255)):
    stack = prepareHistData(img, dst)
    bin_w = dst.shape[1] / 255

    # bruh this python loop
    for i in range(256):
        cv.rectangle(dst, (np.int32(stack[i][0] - bin_w/2), np.int32(stack[i][1])),
                     (np.int32(stack[i][0] + bin_w/2), np.int32(dst.shape[0])), color, -1)


def plotLineHistogram(img, dst, color=(0, 255, 255)):
    stack = prepareHistData(img, dst)

    cv.polylines(dst, [np.int32(stack)], False, color)


In [24]:
img = cv.imread('../imgs/an/test/b1.jpg')
#img = cv.resize(img, (200, 200))
img = cv.cvtColor(img, cv.COLOR_BGR2HSV)[:,:,0]

data  = cv.calcHist([img], [0], None, [256], [0, 256]) #img.ravel()

histogram_canvas = np.zeros((200, 256, 3), np.uint8)
plotBinHistogram(img, histogram_canvas)

while True:
    cv.imshow("histogram", histogram_canvas)
    k = cv.waitKey(0) & 0xFF
    if k == 27:
        break

In [25]:
cv.destroyAllWindows()

In [48]:
size = 200
histSize = 256
histRange = (0, 256) # the upper boundary is exclusive
accumulate = False

def load_img(path):
    img = cv.imread(path)
    #img = cv.resize(img, (size, size))
    img = cv.cvtColor(img, cv.COLOR_BGR2HSV)[:,:,0]
    
    data  = cv.calcHist([img], [0], None, [256], [0, 256]) #img.ravel()

    data = data[:, 0]
    
    return data

In [49]:
X = []
y = []

for f in os.listdir('../imgs/an/badger'):
    data = load_img('../imgs/an/badger/' + f)
    X.append(data)
    y.append(0)
    # print(data)

for f in os.listdir('../imgs/an/chipmunk'):
    data = load_img('../imgs/an/chipmunk/' + f)
    X.append(data)
    y.append(1)
    # print(data)

X = np.asarray(X)
y = np.asarray(y)

print(X.shape)

(15, 256)


In [50]:
pca = PCA(n_components=2)
pca.fit(X)
X_embedded = X# pca.transform(X)# tsne.fit_transform(X)

In [51]:
clf  = GradientBoostingClassifier(n_estimators=200, learning_rate=1.0, max_depth=2, random_state=0)
clf.fit(X, y)

GradientBoostingClassifier(learning_rate=1.0, max_depth=2, n_estimators=200,
                           random_state=0)

In [52]:
print(y)

[0 0 0 0 0 0 0 0 0 1 1 1 1 1 1]


In [53]:
print(clf.score(X, y))

1.0


In [ ]:
def id_to_label(id):
    return 'badger' if id == 0 else 'chipmuck'

In [55]:
test_X = np.asarray([load_img('../imgs/an/test/b2.jpg'), load_img('../imgs/an/test/c1.jpg')])
test_b = test_X#pca.transform(test_X)
clf.predict(test_b)

array([0, 1])